In [42]:
import pandas as pd
import os
import fitz #para número de páginas pdf
import tabula #para obtener datos del pdf
import subprocess

In [43]:
input = r'C:\David\Desarrollos\02_Procesos\09_Control_de_Saldos\02_Python\01_Input'
output = r'C:\David\Desarrollos\02_Procesos\09_Control_de_Saldos\02_Python\02_Output'

In [44]:
archivos_xls = [file.name for file in os.scandir(input) if file.name.endswith('.xls')]
archivos_pdf = [file.name for file in os.scandir(input) if file.name.endswith('.pdf')]

In [45]:
def obtener_numero_paginas_pdf(archivo):
    pdf_document = fitz.open(archivo)
    return pdf_document.page_count

In [46]:
def tratar_df_brutos_de_pdf(max_proc, contable_null, df):
    print(max_proc, contable_null)

    if max_proc == 5 and not contable_null:
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE']]
        df_columnas_por_ubicacion = df.iloc[:, [1, -2]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc == 5 and contable_null:
        df = df.drop(columns = ['CONTABLE'])
        df = df.rename(columns = {df.columns[-1]: 'CONTABLE'})
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE']]
        df_columnas_por_ubicacion = df.iloc[:, [1, -2]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc != 5 and not contable_null:
        df['Columna 2'] = df['PROC.'].str[5:]
        df['PROC.'] = df['PROC.'].str[:5]
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE', 'Columna 2']]
        df_columnas_por_ubicacion = df.iloc[:, [-3]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc != 5 and contable_null:
        #v Inicio de parte no testeada por caso no presentado (se incluye porque el caso en el que max_prox == 5 y contable_null es True sí se ha dado)
        df = df.drop(columns = ['CONTABLE'])
        df = df.rename(columns = {df.columns[-1]: 'CONTABLE'})     
        #^ Fin de parte no testeada por caso no presentado        
        df['Columna 2'] = df['PROC.'].str[5:]
        df['PROC.'] = df['PROC.'].str[:5]
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE', 'Columna 2']]
        df_columnas_por_ubicacion = df.iloc[:, [-3]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)

In [47]:
def generar_dataframe_desde_pagina_pdf(archivo, pagina):
    if pagina == 1:
        df = tabula.read_pdf(archivo, pages=f'{pagina}', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
        max_proc = df['PROC.'].str.len().max()
        contable_null = (pd.isna(df['CONTABLE'].unique())[0] == True)
        df = tratar_df_brutos_de_pdf(max_proc, contable_null, df)
        df = df.rename(columns = {df.columns[-2]: 'DESCRIPCIÓN', df.columns[-1]: 'CARGO_ABONO', 'PROC.': 'FECHA', 'CONTABLE': 'SALDO CONTABLE'})        
        return df[['FECHA', 'DESCRIPCIÓN', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CARGO_ABONO', 'SALDO CONTABLE']]
    
    if pagina > 1:
        df = tabula.read_pdf(archivo, pages=f'{pagina}', stream=True, encoding='ISO-8859-1', area = (195, 0, 842, 595))[0]
        df = df[~(df['PROC.'].isna())] #Lee siempre la primera línea (no encabezados) como nulo
        max_proc = df['PROC.'].str.len().max()
        contable_null = (pd.isna(df['CONTABLE'].unique())[0] == True)
        df = tratar_df_brutos_de_pdf(max_proc, contable_null, df)
        df = df.rename(columns = {df.columns[-2]: 'DESCRIPCIÓN', df.columns[-1]: 'CARGO_ABONO', 'PROC.': 'FECHA', 'CONTABLE': 'SALDO CONTABLE'})  
        return df[['FECHA', 'DESCRIPCIÓN', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CARGO_ABONO', 'SALDO CONTABLE']]
        

In [48]:
def generar_dataframe_desde_pdf(archivo):
    ruta_archivo = os.path.join(input, archivo)
    numero_paginas_pdf = obtener_numero_paginas_pdf(ruta_archivo)
    dfs_temp = []
    #Es numero_paginas_pdf - 1 porque la última página de los pdf siempre muestra información no relevante
    for pagina in range(numero_paginas_pdf - 1):
        #Es pagina + 1 porque el pdf empieza desde página 1 y el range empieza desde 0
        if pagina == 0:
            df_temp_1 = generar_dataframe_desde_pagina_pdf(ruta_archivo, pagina + 1)
            dfs_temp.append(df_temp_1)
        else:
            df_temp_2 = generar_dataframe_desde_pagina_pdf(ruta_archivo, pagina + 1)
            dfs_temp.append(df_temp_2)
    df = pd.concat(dfs_temp)
    return df

In [49]:
def corregir_xls(archivo):

    # Definir el script de PowerShell como una cadena de texto
    powershell_script = f"""
    $OriginalFilePath = "{input + '\\' + archivo}"
    $NewFilePath = "{input + '\\' + archivo + 'x'}"
    $Excel = New-Object -ComObject Excel.Application
    $Excel.Visible = $false
    $Workbook = $Excel.Workbooks.Open($OriginalFilePath)
    $Workbook.SaveAs($NewFilePath, 51)
    $Workbook.Close()
    $Excel.Quit()
    [System.Runtime.Interopservices.Marshal]::ReleaseComObject($Excel) | Out-Null
    [System.Runtime.Interopservices.Marshal]::ReleaseComObject($Workbook) | Out-Null
    """

    # Ejecutar el script de PowerShell
    subprocess.run(["powershell", "-Command", powershell_script])

    return archivo + 'x'

In [50]:
def generar_dataframe_desde_xls(archivo):
    nuevo_archivo = corregir_xls(archivo)
    df = pd.read_excel(os.path.join(input, nuevo_archivo), skiprows = 24)
    return df[df['FECHA OPER.'].str.len() == 6]

In [51]:
for archivo in archivos_pdf:
    empresa_moneda = archivo[-11:-4]

    match empresa_moneda:
        case 'BPO PEN':
            df_BCP_BPO_PEN = generar_dataframe_desde_pdf(archivo)
        case 'BPO USD':
            df_BCP_BPO_USD = generar_dataframe_desde_pdf(archivo)
        case 'SDP USD':
            df_BCP_SDP_USD = generar_dataframe_desde_pdf(archivo)
        case 'SDP PEN':
            df_BCP_SDP_PEN = generar_dataframe_desde_pdf(archivo)
        case _:
            print('Se ha detectado un caso desconocido de EMPRESA_MONEDA en los archivos pdf')

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


5 False
5 False
40 False
5 False
5 False
5 False
5 False
5 False
40 False
5 False
5 False
5 False
5 False
5 False
31 False
5 False
5 True
5 False


In [52]:
for archivo in archivos_xls:
    empresa_moneda = archivo[-11:-4]

    match empresa_moneda:
        case 'SDP PEN':
            df_BBVA_SDP_PEN = generar_dataframe_desde_xls(archivo)
        case 'SDP USD':
            df_BBVA_SDP_USD = generar_dataframe_desde_xls(archivo)
        case _:
            print('Se ha detectado un caso desconocido de EMPRESA_MONEDA en los archivos xls')


In [ ]:
df_BCP_BPO_PEN
df_BCP_BPO_USD
df_BCP_SDP_USD
df_BCP_SDP_PEN
df_BBVA_SDP_PEN
df_BBVA_SDP_USD